# Use Selenium web crawler to auto-download daily reports

In this python script, I used selenium web crawler to download reports from external website with given username and password and perform further analysis. This script is then set to run at certain time with Windows task scheduler. I wrote five functions to retrieve different reports and perform corresponding analysis for different business objectives.

### Modules and path settings for the download

First, I imported selenium and downloaded its required webdriver for web-crawling. I also jotted down the engineering memo for future maintenance.   

In [1]:
"""
Engineering Memo:
1. Change login information if needed
2. Chromedriver needs to be updated if needed
3. The old file will be replaced if running the whole code more than one time in the same day
4. Below five function can be executed separately

"""

from selenium import webdriver
import time
import os
from pathlib import Path
import datetime
import glob
import pandas as pd
import numpy as np

In [2]:
home = str(Path.home())

# path1 = chromedriverpath
# path2 = download location
path1 = os.path.join(home, 'Documents', 'Python', 'chromedriver.exe')
path2 = os.path.join(home, 'Documents', 'Python', 'Webcrawling')

# chrome
chromeOptions = webdriver.ChromeOptions()

#Directory
prefs = {"download.default_directory" : path2}
chromeOptions.add_experimental_option("prefs",prefs)

chromeOptions.add_argument("--disable-notifications")

chrome = webdriver.Chrome(path1, options=chromeOptions)

C:\Users\hout\AppData\Local\Temp/ipykernel_32472/4017793452.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome = webdriver.Chrome(path1, options=chromeOptions)


### Function1 : Download Inventory Aging reports - SiteA

In [7]:
def Download_Schenkeragingreport():

    chrome.get(#Address)

    # login
    username = chrome.find_element_by_name("user")
    username.click()
    username.send_keys(#UserName)

    password = chrome.find_element_by_name("password")
    password.send_keys(#Password)

    login = chrome.find_element_by_name("image1")
    login.click()
    time.sleep(1)

    # CSview
    CsView = chrome.find_element_by_css_selector("body > form > table > tbody > tr:nth-child(8) > td > a > img")
    CsView.click()
    time.sleep(1)

    # Inventoryaging
    Invaging = chrome.find_element_by_css_selector("td:nth-child(1) > table > tbody > tr:nth-child(4) > td > a")
    Invaging.click()
    time.sleep(1)

    #Download
    downloadI = chrome.find_element_by_css_selector("table > tbody > tr.Criteria2 > td:nth-child(6) > input")
    downloadI.click()

### Function2 : Download Receipt Summary reports - SiteA

In [ ]:
def Download_SchenkerReceiptSummary():

    chrome.get(#Address)

    # login
    username = chrome.find_element_by_name("user")
    username.click()
    username.send_keys(#UserName)

    password = chrome.find_element_by_name("password")
    password.send_keys(#Password)

    login = chrome.find_element_by_name("image1")
    login.click()

    # CSview
    CsView = chrome.find_element_by_css_selector("body > form > table > tbody > tr:nth-child(8) > td > a > img")
    CsView.click()
    time.sleep(1)

    # ReceiptSummary
    RcptSummary = chrome.find_element_by_css_selector("td:nth-child(2) > table > tbody > tr:nth-child(2) > td > a")
    RcptSummary.click()
    time.sleep(1)

    # Shipstartdate
    Shipstartdate = chrome.find_element_by_css_selector(
        "table.criteria > tbody > tr.criteria2 > td:nth-child(6) > a:nth-child(2) > img")
    Shipstartdate.click()
    time.sleep(1)

    #This is to define css selector place for the first date of every month
    def weekdayoffirstdate():
        today = datetime.date.today().replace(day=1)
        day = today.weekday()
        if day in (0, 1, 2, 3, 4):
            day += 2
            code = 'td:nth-child(' + str(day) + ')'
        elif day == 5:
            code = 'td.cal-DayCell'

        elif day == 6:
            day = 1
            code = 'td:nth-child(' + str(day) + ')'
        return code

    # Date_first
    date1 = chrome.find_element_by_css_selector(
        "tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(2) > " + weekdayoffirstdate() + ' > a')
    date1.click()
    time.sleep(1)

    # Download
    downloadR = chrome.find_element_by_css_selector("table.criteria > tbody > tr.criteria2 > td:nth-child(7) > input")
    downloadR.click()

### Function3 : Download Inventory Aging reports - SiteB

In [ ]:
def Download_Jusdaagingreport():

    chrome.get(#Address)

    # login
    username = chrome.find_element_by_id("textUID")
    username.click()
    username.send_keys(#UserName)

    password = chrome.find_element_by_id("textPWD")
    password.send_keys(#Password)

    login = chrome.find_element_by_id("submitBtn")
    login.click()

    #Switch frame
    chrome.switch_to.frame(chrome.find_element_by_css_selector('html > frameset > frameset > frameset > frame:nth-child(1)'))

    Inventory = chrome.find_element_by_id('webfx-tree-object-9-plus')
    Inventory.click()

    Invaging = chrome.find_element_by_id('webfx-tree-object-11-anchor')
    Invaging.click()

    # Switch frame
    chrome.switch_to.default_content()
    chrome.switch_to.frame(chrome.find_element_by_name('main'))

    GO = chrome.find_element_by_css_selector('table:nth-child(6) > tbody > tr > td:nth-child(2) > a')
    GO.click()

    Export = chrome.find_element_by_css_selector('table:nth-child(7) > tbody > tr > td:nth-child(2) > a')
    Export.click()

### Function4 : Perform basic data analysis with pandas for the downloaded reports

In [ ]:
def Transform_ReceiptSummary_LCD():

    #Path for old receiptsummary csv
    oldpath_schenker_receipt = glob.glob(os.path.join(path2, 'receipt*'))

    #Filter to SOI and LCD
    df_receipt = pd.read_csv(oldpath_schenker_receipt[0])
    df_receipt['commodity'] = df_receipt['commodity'].str.strip()
    df_receipt = df_receipt.loc[(df_receipt['commodity'] == 'LCD')]
    df_receipt['owner'] = df_receipt['owner'].str.strip()
    df_receipt2 = df_receipt.loc[df_receipt['owner'] == 'SOI']

    #Pivot for LCD
    df_table = pd.pivot_table(df_receipt2, values='qty_received', index=['hp_part_no', 'descr'], aggfunc=np.sum)

    #Fill merge cells to previous value
    df_table = df_table.fillna(method='ffill')

    #Reset index
    df_table = df_table.reset_index()

    #df_all = empty df
    df_all = pd.DataFrame(columns = ['hp_part_no', 'descr', 'qty_received'])
    #df_lookup = df for lookup table
    df_lookup = pd.read_excel(os.path.join(path2, 'Part number_Lookup_table.xlsx'))

    #Match df_lookup with df_table and write to df_all
    for index, row in df_lookup.iterrows():
        df = df_table.loc[df_table['hp_part_no'].str.startswith(str(row['Part number'])), :]
        df_all = df_all.append(df, ignore_index=True)

    #Export
    Current_Date = datetime.datetime.today().strftime('%Y%m%d')
    new_path = os.path.join(path2, 'LCD_Receipt Summary-' + Current_Date + '.xlsx')
    writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
    df_receipt.to_excel(writer, sheet_name='LCD_ReceiptSummary')
    df_table.to_excel(writer, sheet_name='Pivot')
    df_all.to_excel(writer, sheet_name='Lookup_Value')
    writer.save()

    #Remove old csv file
    os.remove(oldpath_schenker_receipt[0])

### Function5 : Change report names for dynamic input

In [ ]:
def ChangeFileName():

    oldpath_jusda = glob.glob(os.path.join(path2, 'Export*'))
    oldpath_schenker = glob.glob(os.path.join(path2, 'inventoryaging*'))

    Current_Date = datetime.datetime.today().strftime('%Y%m%d')

    newpath_jusda = os.path.join(path2, 'Jusda Inventory -' + Current_Date + '.xls')
    newpath_schenker = os.path.join(path2, 'Schenker Inventory -' + Current_Date + '.xls')

    os.replace(oldpath_jusda[0], newpath_jusda)
    os.replace(oldpath_schenker[0], newpath_schenker)

#### Run Functions

In [ ]:
#Run
Download_Schenkeragingreport()
Download_SchenkerReceiptSummary()
Download_Jusdaagingreport()
time.sleep(15)
Transform_ReceiptSummary_LCD()
ChangeFileName()